# 1. 기존 방법론에 대한 비판
**LSA**는 DTM/TF-IDF 빈도수 카운트 행렬을 truncated SVD해 잠재 의미 도출   
전체적 통계 정보를 고려하나, 단어 의미 유추 작업에는 성능이 떨어짐   
**Word2Vec**은 예측값과 실제값 오차를 손실 함수로 줄여나가며 학습   
유추 작업에는 뛰어나나, 윈도우에서만 주변 단어를 고려해 전체적 통계 정보를 반영 못함   
**Glove**는 카운트와 예측 기반을 모두 사용

# 2.윈도우 기반 동시 등장 행렬(Window based Co-occurence Matrix)
전체 단어 집합으로 행렬 구성   

카운트|I|like|enjoy|deep|learning|NLP|flying
:--|:--|:--|:--|:--|:--|:--|:--
I|0|2|1|0|0|0|0
like|2|0|0|1|0|1|0
enjoy|1|0|0|0|0|0|1
deep|0|1|0|0|1|0|0
learning|0|0|0|1|0|0|0
NLP|0|1|0|0|0|0|0
flying|0|0|1|0|0|0|0

# 3. 동시 등장 확률(Co-occurence Probability)
동시 등장 행렬에서 특정 단어의 전체 등장 횟수와, 그 특정 단어가 등장할 때 어떤 단어가 등장한 횟수를 계산한 조건부 확률

동시 등장 확률과 크기 관계 비|k=solid|k=gas|k=water|k=fashion
:--|:--|:--|:--|:--
P(ice가 등장할 때 k의 확률)|큰 값|작은 값|큰 값|작은 값
P(steam이 등장할 때 k의 확률)|작은 값|큰 값|큰 값|작은 값
P(ice가 등장할 때 k의 확률) / P(steam이 등장할 때 k의 확률)|큰 값|작은 값|1에 가까움|1에 가까움

# 4. 손실 함수(Loss function)
GloVe는 임베딩된 중심 단어와 주변 단어의 내적이, 전체 corpus에서 동시 등장 확률임
$$w_{i} \cdot \tilde{w_{k}} \approx P(k\,|\,i) = P_{ik}$$

벡터들로 함수 F 적용시 동시 등장 확률 비가 나온다는 초기 식 가정
$$F(w_{i}, w_{j}, \tilde{w_{k}}) = \frac{P_{ik}}{P_{jk}}$$

F는 동시 등장 확률 비를 벡터 공간에 인코딩하는 목적
$$F(w_{i} - w_{j}, \tilde{w_{k}}) = \frac{P_{ik}}{P_{jk}}$$

좌변은 벡터고 우변은 스칼라임을 성립해주는 내적   
(정리하자면 선형 공간에서 단어 의미 관계를 표현하고자 뺄셈과 내적 사용)
$$F((w_{i} - w_{j})^{T}\tilde{w_{k}}) = \frac{P_{ik}}{P_{jk}}$$   

이때 F는 실수 덧셈과 양수 곱셈에서 준동형(homomorphism)을 만족   
$$F(a + b) = F(a)F(b),\,\forall{a},\,b \in \mathbb{R}$$
$$F(v_{1}^{T}v_{2} + v_{3}^{T}v_{4}) = F(v_{1}^{T}v_{2})F(v_{3}^{T}v_{4}),\,\forall{v_{1},\,v_{2},\,v_{3},\,v_{4}} \in V$$
$$F(v_{1}^{T}v_{2} - v_{3}^{T}v_{4}) = \frac{F(v_{1}^{T}v_{2})}{F(v_{3}^{T}v_{4})},\,\forall{v_{1},\,v_{2},\,v_{3},\,v_{4}} \in V$$

이를 GloVe에 적용
$$F((w_{i} - w_{j})^{T}\tilde{w_{k}}) = \frac{F(w_{i}^{T}\tilde{w_{k}})}{F(w_{j}^{T}\tilde{w_{k}})} = \frac{P_{ik}}{P_{jk}}$$

이를 만족하는 F는 exponential function
$$exp(w_{i}^{T}\tilde{w_{k}} - w_{j}^{T}\tilde{w_{k}}) = \frac{exp(w_{i}^T\tilde{w_{k}})}{exp(w_{j}^T\tilde{w_{k}})}$$
$$exp(w_{i}^{T}\tilde{w_{k}}) = P_{ik} = \frac{X_{ik}}{X_{i}}$$
$$w_{i}^{T}\tilde{w_{k}} = logP_{ik} = log(\frac{X_{ik}}{X_{i}}) = logX_{ik} - logX_{i}$$

각 임베딩 벡터의 위치를 바꾸어도 식이 성립해야 하는데 로그항이 걸림돌로 작용하므로, 로그항을 편향으로 대체
$$w_{i}^{T}\tilde{w_{k}} + b_{i} + \tilde{b_{k}} = logX_{ik}$$

손실 함수는 이 차이를 최소화하는 방향으로 일반화    
로그항이 0이 될 수 있어 log(1+X)로 사용하기도 함
$$Loss function = \sum_{m,n=1}^{V}(w_{m}^{T} + b_{m} + \tilde{b_{n}} - logX_{mn})^{2}$$
   
희소 행렬에서 낮은 빈도수에 가중치를 주는 weight function 도입, X가 커질 때 지나친 가중치를 막고자 최댓값 설정   
$$f(x) = min(1, (\frac{x}{x_{max}})^{\frac{3}{4}})$$
![weight function](https://wikidocs.net/images/page/22885/%EA%B0%80%EC%A4%91%EC%B9%98.PNG "weight function")

최종 손실 함수
$$Loss function = \sum_{m,n=1}^{V}\,f(X_{mn})(w_{m}^{T}\tilde{w_{n}} + b_{m} + \tilde{b_{n}} - logX_{mn})^{2}$$

# 5. GloVe 훈련시키기

In [1]:
import nltk
nltk.download('punkt')

import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml",
                           filename="ted_en-20160408.xml")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


('ted_en-20160408.xml', <http.client.HTTPMessage at 0x1e5cecf0e20>)

In [2]:
# 전처리
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

parse_text = '\n'.join(target_text.xpath('//content/text()'))
content_text = re.sub(r'\([^)]*\)', '', parse_text)

sent_text = sent_tokenize(content_text)

normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

result = [word_tokenize(sentence) for sentence in normalized_text]

In [3]:
from glove import Corpus, Glove

# 훈련 데이터로부터 동시 등장 행렬 생성
corpus = Corpus()
corpus.fit(result, window=5)

glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


In [4]:
model_result1 = glove.most_similar("man")
print(model_result1)

[('woman', 0.9648368487300606), ('guy', 0.87853441458273), ('girl', 0.8534987437931901), ('young', 0.846071380470317)]


In [5]:
model_result2 = glove.most_similar("boy")
print(model_result2)

[('girl', 0.9476923851537493), ('woman', 0.8419994258815078), ('kid', 0.8361017051820097), ('man', 0.8279171755525907)]


In [6]:
model_result3 = glove.most_similar("university")
print(model_result3)

[('harvard', 0.888141053853674), ('mit', 0.8571900561051062), ('cambridge', 0.8353162335157012), ('stanford', 0.8343890928881722)]


In [7]:
model_result4 = glove.most_similar("water")
print(model_result4)

[('clean', 0.8436805363087417), ('air', 0.8276533018109319), ('electricity', 0.818030617567804), ('fresh', 0.8173633565603985)]


In [8]:
model_result5 = glove.most_similar("physics")
print(model_result5)

[('chemistry', 0.8938172020985033), ('economics', 0.8825295341379408), ('beauty', 0.8640229385013825), ('mathematics', 0.8553610296644241)]


In [9]:
model_result6 = glove.most_similar("muscle")
print(model_result6)

[('tissue', 0.8488022951766478), ('nerve', 0.836366071812534), ('stem', 0.7888055281526625), ('channel', 0.7717473953205604)]


In [10]:
model_result7 = glove.most_similar("clean")
print(model_result7)

[('water', 0.8436805363087417), ('fresh', 0.8321380109071955), ('wind', 0.8010232560853364), ('heat', 0.7796518615106788)]
